In [142]:
from importlib import reload
import load_data
reload(load_data)
from load_data import *

import numpy as np

In [6]:
url = "https://datasets.imdbws.com/"
filenames = ["name.basics.tsv.gz",
             "title.akas.tsv.gz",
             "title.basics.tsv.gz",
             "title.crew.tsv.gz",
             "title.episode.tsv.gz",
             "title.principals.tsv.gz",
             "title.ratings.tsv.gz"
]

output_paths = download_file(url, filenames, "data/")

# Pierwszy rzut oka na dane

Pierwsza ramka:

In [141]:
path_to_file = "data/name.basics.tsv.gz"
df = load_to_dataframe(path_to_file)

AttributeError: type object 'Int16Dtype' has no attribute 'descr'

In [ ]:
df.head()

Druga ramka:

In [149]:
path_to_file = "data/title.akas.tsv.gz"
df = load_to_dataframe(path_to_file)

ParserError: Error tokenizing data. C error: Unknown error in IO callback

In [ ]:
df = df.drop(columns=["language", "types", "attributes"])

KeyboardInterrupt: 

In [134]:
df["title"].astype(str).str.lower()

KeyboardInterrupt: 

In [126]:
df.loc[df["titleId"] == 'tt0017938']

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
72318,tt0017938,1,<built-in method lower of str object at 0x0000...,<NA>,<NA>,original,<NA>,1
72319,tt0017938,2,<built-in method lower of str object at 0x0000...,DE,<NA>,<NA>,<NA>,0
72320,tt0017938,3,<built-in method lower of str object at 0x0000...,ES,<NA>,<NA>,<NA>,0
72321,tt0017938,4,<built-in method lower of str object at 0x0000...,EC,<NA>,imdbDisplay,<NA>,0
72322,tt0017938,5,<built-in method lower of str object at 0x0000...,FR,<NA>,imdbDisplay,<NA>,0
72323,tt0017938,6,<built-in method lower of str object at 0x0000...,JP,ja,imdbDisplay,<NA>,0
72324,tt0017938,7,<built-in method lower of str object at 0x0000...,PL,<NA>,<NA>,<NA>,0
72325,tt0017938,8,<built-in method lower of str object at 0x0000...,XWW,en,<NA>,<NA>,0
72326,tt0017938,9,<built-in method lower of str object at 0x0000...,SUHH,ru,imdbDisplay,<NA>,0


In [107]:
# wydostajemy kandydatów na filmy z których odzyskamy kraje
# zgodnie z uwagą w PDFie od prowadzących
df_think = df.loc[:, ['titleId', 'title', 'region', 'isOriginalTitle']].query(
    "isOriginalTitle == 0"
).merge(
    df.loc[:, ['titleId', 'title', 'isOriginalTitle']].query("isOriginalTitle == 1"),
    on="titleId",
    how="left",
    suffixes=("", "_y")
).query(
    "title == title_y"
).drop(
    labels=["title_y", "isOriginalTitle", "isOriginalTitle_y"],
    axis=1
)

In [108]:
df_think[df_think["titleId"] == 'tt0001498']

,titleId,title,region
3427,tt0001498,The Battle of Trafalgar,EC
3428,tt0001498,The Battle of Trafalgar,GB
3429,tt0001498,The Battle of Trafalgar,US


In [110]:
# grupujemy po ID tytułu
# potem zliczamy ile było przypisanych regionów
# tam, gdzie był jeden, daje się zidentyfikować jednoznacznie
ambig_index = df_think.loc[:, ["titleId", "region"]].groupby(
    by=["titleId"]
).count().where(
    lambda region: region > 1
).dropna().index

df_with_known_countries = df_think.loc[~df_think.titleId.isin(list(ambig_index))]

In [112]:
df_with_known_countries[df_with_known_countries["region"] == 'EC']

,titleId,title,region
40700,tt0015039,Kinoglaz,EC
42303,tt0015361,Stachka,EC
48902,tt0016847,Faust: Eine deutsche Volkssage,EC
49872,tt0017048,Kurutta ippêji,EC
54644,tt0017938,La glace à trois faces,EC
...,...,...,...
37661612,tt9826286,Kamenák,EC
37752292,tt9876760,80's in the Sand,EC
37758139,tt9880498,El Fútbol del Cerro,EC
37761046,tt9881986,Hentai danchizuma,EC


In [15]:
# utrata danych: 26.6%
1 - df_with_known_countries.shape[0]/df_think.shape[0]

0.2663587633217803

In [18]:
# Teraz do każdego tytułu w 'df' przypisaliśmy kraj, jeśli się da
# jeśli nie, mamy 'NaN' w kolumnie 'region'
df = df.loc[df.isOriginalTitle == 1].loc[:, ["titleId", "title"]].merge(
            df_with_known_countries.loc[:, ["titleId", "region"]],
            how="left",
            on="titleId")

Trzecia ramka:

In [52]:
path_to_file = "data/title.basics.tsv.gz"
df3 = load_to_dataframe(path_to_file)
df3.head()

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,<NA>,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,<NA>,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,<NA>,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,<NA>,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,<NA>,1,"Comedy,Short"


In [55]:
df3 = df3.loc[:, ["tconst", "titleType"]]
df3.head()

,tconst,titleType
0,tt0000001,short
1,tt0000002,short
2,tt0000003,short
3,tt0000004,short
4,tt0000005,short


Czwarta ramka:

In [ ]:
path_to_file = "data/title.crew.tsv.gz"
df = load_to_dataframe(path_to_file)
df.head()

Piąta ramka:

In [ ]:
path_to_file = "data/title.episode.tsv.gz"
df = load_to_dataframe(path_to_file)
df.head()

Szósta ramka:

In [ ]:
path_to_file = "data/title.principals.tsv.gz"
df = load_to_dataframe(path_to_file)
df.head()

Siódma ramka:

In [21]:
path_to_file = "data/title.ratings.tsv.gz"
df = load_to_dataframe(path_to_file)
df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2058
1,tt0000002,5.6,277
2,tt0000003,6.5,2024
3,tt0000004,5.4,179
4,tt0000005,6.2,2791


# Task 1
Quality of movies by country

In [20]:
df_countries = df.copy()

In [63]:
df_analysis = df_countries.merge(
        df,
        how="left",
        left_on="titleId",
        right_on="tconst"
).merge(
        df3,
        how="left",
        left_on="titleId",
        right_on="tconst"
).query(
        "titleType == 'movie'"
).drop(
        columns=["tconst_x", "tconst_y", "titleType"]
).dropna()

In [64]:
df_analysis.sort_values(ascending=False, by="numVotes")

,titleId,title,region,averageRating,numVotes
235039,tt0245429,Sen to Chihiro no kamikakushi,PH,8.6,856653.0
332287,tt0347149,Hauru no ugoku shiro,CA,8.2,453576.0
3199327,tt1409024,Men in Black³,US,6.8,389083.0
8925358,tt5700672,Busanhaeng,KR,7.6,261857.0
287073,tt0299977,Ying xiong,EC,7.9,188070.0
...,...,...,...,...,...
6831431,tt28449981,Hridita,BD,5.8,5.0
6826158,tt28434969,Estols,ES,4.2,5.0
6825839,tt28434315,Beyond Tradition: Power of Natural Voice,CH,7.0,5.0
267194,tt0279167,Karaiyellam Shenbagappoo,IN,6.6,5.0


Pytanie: jaką miarę przyłożyć do oceny jakości?
Pomysły:
1. Sortować malejąco po ocenie
2. Sortować malejąco po ocenie, ale odfiltrować filmy z mniej niż $K$ (parametr) ocenami
3. Stworzyć własny indeks, np. $\text{rating} = \text{averageRating}\cdot\ln{(\text{numVotes})}$

Pomysł 1 - nie działa:

In [65]:
df_analysis.sort_values(by="averageRating", ascending=False).head(10)

,titleId,title,region,averageRating,numVotes
2301271,tt12413918,Zayed's Antarctic Lights,AE,10.0,9.0
7760283,tt32384376,Historias de cuarentena,DO,10.0,5.0
2324317,tt12458850,MAMA!,US,10.0,7.0
4965441,tt1918943,Max Ernst Hanging,US,10.0,8.0
2688578,tt13145694,The Church: Pillar and Ground of the Truth,US,10.0,6.0
7025245,tt29259789,Sisters & The Shrink 4,US,10.0,8.0
5488368,tt21403948,Rijali,TZ,10.0,6.0
1216178,tt10449358,Kaputol,PH,10.0,8.0
5368597,tt21111272,Stigma - Rusbruk i Norge,NO,10.0,12.0
7539111,tt31512788,Manadarasi,IN,10.0,6.0


Pomysł 2 - działa nieźle 

In [66]:
K = 10000
df_analysis.loc[df_analysis["numVotes"] > K].sort_values(by="averageRating", ascending=False).head(10)

,titleId,title,region,averageRating,numVotes
243005,tt0253828,Tosun Pasa,IT,8.9,24687.0
241839,tt0252597,Kibar Feyzo,TR,8.7,17415.0
193408,tt0201368,Zügürt Aga,TR,8.7,16446.0
242802,tt0253614,Saban Oglu Saban,EC,8.7,18812.0
241734,tt0252489,Hababam Sinifi Tatilde,TR,8.6,18905.0
235039,tt0245429,Sen to Chihiro no kamikakushi,PH,8.6,856653.0
57501,tt0058625,Suna no onna,EC,8.5,22900.0
58413,tt0059550,Operatsiya 'Y' i drugie priklyucheniya Shurika,EC,8.4,14910.0
49717,tt0050634,Letyat zhuravli,EC,8.3,19940.0
57763,tt0058888,Akahige,EC,8.3,21261.0


Pomysł 3 -

In [73]:
df_analysis.loc[:, ["rating"]] = df_analysis["averageRating"] * np.log(df_analysis["numVotes"])

In [86]:
first_100 = df_analysis.sort_values(by="rating", ascending=False).head(100)
first_100

,titleId,title,region,averageRating,numVotes,rating
235039,tt0245429,Sen to Chihiro no kamikakushi,PH,8.6,856653.0,117.482779
332287,tt0347149,Hauru no ugoku shiro,CA,8.2,453576.0,106.804329
54569,tt0055630,Yôjinbô,EC,8.2,132042.0,96.685178
287073,tt0299977,Ying xiong,EC,7.9,188070.0,95.942099
52193,tt0053198,Les quatre cents coups,CA,8.1,128567.0,95.290064
...,...,...,...,...,...,...
391761,tt0408777,Die fetten Jahre sind vorbei,DE,7.4,32412.0,76.858502
10257605,tt8652818,Gekijôban Vaioretto Evâgâden,JP,8.3,10454.0,76.814342
10793090,tt9817070,Metri shesh va nim,IR,7.8,18825.0,76.774940
56722,tt0057831,Alexis Zorbas,ZA,7.6,24206.0,76.717104


In [85]:
first_100.loc[:, ["region", "rating"]].groupby(by="region").count()

,rating
region,
AE,4
BD,1
BR,1
CA,7
CH,1
DE,2
DK,2
EC,42
FI,1
